In [2]:
from dictionary_learning.trainers.top_k import AutoEncoderTopK
import torch
from utils import MODELS_DIR, INPUT_DATA_DIR
from transformers import set_seed
from transformers import MusicgenForConditionalGeneration, AutoProcessor
from IPython.display import Audio, display

device = "cuda:1"
model_name = "facebook/musicgen-medium"
max_tokens = 200
base_dir = INPUT_DATA_DIR / "music-bench" / "datashare-instruments"
model_sr = 32000

In [3]:
model = MusicgenForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()
processor = AutoProcessor.from_pretrained(model_name)
ae = AutoEncoderTopK.from_pretrained(
    MODELS_DIR / "medium-sae-trivial-medium-sae-ee3b" / "16" / "trainer_0" / "checkpoints" / "ae_71100.pt"
).to(device)

Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summ

In [16]:
val = -2
name = "classicguitar_activation"
with torch.no_grad():
    raggae_out = torch.load(INPUT_DATA_DIR / "unlearn" / f"{name}.pt")
    raggae_mu = torch.load(INPUT_DATA_DIR / "unlearn" / f"{name}_mu.pt")
    q = 0.99
    thr = torch.quantile(raggae_out.float(), q).cpu()
    x = raggae_out.detach().cpu()
    # x[x > thr] = -100
    x[[118, 256, 1910, 4636, 4908, 5853, 1563, 1932, 2254, 2251, 549, 5033, 5132, 3920]] = -100
    ind = x.topk(1).indices
    mask = torch.zeros(x.shape[0])
    mask[ind] = 1
    intervention_mask = raggae_mu.detach().cpu()
    intervention_mask[~mask.bool()] = 1 / val
    intervention_mask *= val
intervention_mask = intervention_mask.to(device)

In [17]:
ind

tensor([1244])

In [20]:
hook_point = model.decoder.get_submodule("model.decoder.layers.16")
y = [None]
i = []
set_seed(42)


def hook(module, input, output):
    # set_seed(42)
    i.append(input)
    z = ae.encode(output[0], use_threshold=True)
    # z[:, :, ind] *= -0.1
    z = intervention_mask * z
    return (ae.decode(z), output[1])


remove_handle = hook_point.register_forward_hook(hook)
inputs = processor(
    text=[
        "A solo classical guitar performance with rich, warm tones, intricate fingerpicking, and expressive dynamics, in the style of traditional Spanish classical music, recorded in a clear studio environment."
        # "Fast-paced classical guitar with vibrant violin, clarinet, and hand percussion, in a cheerful Mediterranean style."
    ]
    * 3,
    padding=True,
    return_tensors="pt",
)
# dataset = load_dataset("sanchit-gandhi/gtzan", split="train", streaming=True)
# sample = next(iter(dataset))["audio"]

# # take the first half of the audio sample
# sample["array"] = sample["array"][: len(sample["array"]) // 2]

# inputs = processor(
#     audio=sample["array"],
#     sampling_rate=sample["sampling_rate"],
#     text=["80s blues track with groovy saxophone"],
#     padding=True,
#     return_tensors="pt",
# )
inputs = {k: v.to(device) for k, v in inputs.items()}
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)
for x in audio_values.detach().cpu():
    display(Audio(x, rate=model.config.sampling_rate))
remove_handle.remove()
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)
for x in audio_values.detach().cpu():
    display(Audio(x, rate=model.config.sampling_rate))